<center>

*******************************************************************************************
    
### BATCH PROCESS
### RENAME FILES
  
##### 10 November 2024

##### Juan Ignacio Mendoza Garay  

*******************************************************************************************

</center>

#### INFORMATION:


* Description:

    Rename all files in a folder preserving the extension.
    Useful to anonymise data files.

* Instructions:

    Edit the values indicated with an arrow like this: <---  
    Comment/uncomment or change values as suggested by the comments.  
    Run the program, close your eyes and hope for the best.  

*******************************************************************************************


In [ ]:
import os
import string
import random
from datetime import datetime

In [ ]:
#data_folder = r'C:\Users\hoppy\Documents\my_files' # <--- full path (prepend r)
common_string = 'processed' # <--- common string (e.g., 'renamed'), empty = no common string

nf_nt = 1 # <--- new file name type:
# 0 = common string and random string (e.g., renamed_LP4ZX.ext)
# 1 = common string, current date and number (e.g., renamed_20241110_1.ext)
# 2 = common string and number (e.g., renamed_1.ext)

In [ ]:
if common_string:
    common_string_p = common_string + '_'
else:
    common_string_p = common_string
    
if nf_nt == 0:
    def ran_str(L=5, cv=string.ascii_uppercase + string.digits):
        return ''.join(random.choice(cv) for _ in range(L))
elif nf_nt == 1:
    c = 0

files_in_folder = os.listdir(data_folder)
for src_fn in files_in_folder:
    
    full_src = data_folder+'\\'+src_fn
    # print(full_src)
    this_n, this_ext  = os.path.splitext(src_fn)
    
    if nf_nt == 0:
        output_fn = common_string_p + ran_str() + this_ext
    elif nf_nt == 1:
        d = datetime.today().strftime('%Y%m%d')
        c += 1
        output_fn = common_string_p + f'{d}_{c}' + this_ext
    elif nf_nt == 2:
        c = c+1
        output_fn = common_string_p + ('%i' % c) + this_ext
    #print(output_fn)
    #print( )
    full_dst = data_folder+'\\'+output_fn
    os.rename(full_src, full_dst)

print('done')